In [ ]:
import pathlib
import sys
sys.path.insert(0, str(pathlib.Path().absolute().parent))

In [ ]:
from linkalike.datasets import movielens100k

data, users, items = movielens100k()
users.drop(columns=['zip_code'], inplace=True)
data['rating'] = (data['rating'].to_numpy() > 3).astype(int)

In [ ]:
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer

kbins = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')

uid = "user_id"
iid = "item_id"

num_cols = users.drop(columns=[uid]).select_dtypes(include=np.number).columns
kbins.fit(users[num_cols])
users[[f"{c}_BIN" for c in num_cols]] = kbins.transform(users[num_cols]).astype(int).astype(str)
users.drop(columns=num_cols, inplace=True)
users

In [ ]:
import networkx as nx

G = nx.DiGraph()
interactions = data[data['rating'] == 1]
elist = [tuple(e)for e in interactions[[uid, iid]].to_numpy()]
G.add_edges_from(elist)

In [ ]:
from linkalike.utils.utils import pairwise_dissimilarities

dist_df = pairwise_dissimilarities(users, verbose=True)
dist_df

